In [ ]:
# ==============================================================================
# OPTIMIZED AI VOICE DETECTOR V7 (Sanitized Indentation)
# ==============================================================================
!pip install -q kaggle librosa soundfile ffmpeg-python gTTS tqdm scikit-learn matplotlib torch torchvision torchaudio

import os
import shutil
import random
import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.amp import autocast, GradScaler
from pathlib import Path
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# CONFIGURATION
CONFIG = {
    'SR': 16000,
    'DURATION': 4.0,
    'N_MELS': 80,
    'HOP_LEN': 512,
    'MAX_SAMPLES': 2000,
    'BATCH_SIZE': 32,
    'EPOCHS': 10,
    'LR': 1e-3,
    'SEED': 42,
    'DATASET_SLUG': 'adarshsingh0903/audio-deepfake-detection-dataset'
}

ROOT_DIR = Path('/content/deepfake_project')
RAW_DIR = ROOT_DIR / 'raw'
MEL_DIR = ROOT_DIR / 'mels'
MODELS_DIR = ROOT_DIR / 'models'

# 1. SETUP & CLEANUP
print("🧹 Cleaning up old data...")
if RAW_DIR.exists(): shutil.rmtree(RAW_DIR)
if MEL_DIR.exists(): shutil.rmtree(MEL_DIR)
for p in [RAW_DIR, MEL_DIR, MODELS_DIR]: p.mkdir(parents=True, exist_ok=True)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Running on: {DEVICE}")

# 2. KAGGLE DOWNLOAD
def setup_kaggle():
    kaggle_path = Path('/root/.kaggle/kaggle.json')
    if kaggle_path.exists(): return True
    try:
        from google.colab import files
        print("📂 Upload kaggle.json if prompted:")
        uploaded = files.upload()
        if 'kaggle.json' in uploaded:
            kaggle_path.parent.mkdir(parents=True, exist_ok=True)
            with open(kaggle_path, 'wb') as f: f.write(uploaded['kaggle.json'])
            os.chmod(kaggle_path, 0o600)
            return True
    except: pass
    return False

if setup_kaggle():
    print(f"⬇️ Downloading Dataset (1GB)...")
    os.system(f"kaggle datasets download -d {CONFIG['DATASET_SLUG']} -p {RAW_DIR} --unzip")
    print("✅ Download Complete.")

# 3. ROBUST DATA SCANNER
def scan_dataset(root):
    data = []
    print(f"🔍 Scanning {root} for audio files...")

    # Walk through folders
    for f in tqdm(root.rglob('*')):
        if f.is_file() and f.suffix.lower() in {'.wav', '.mp3', '.flac'}:
            path_str = str(f).lower()

            label = None
            if 'real' in path_str or 'bonafide' in path_str:
                label = 0
            elif 'fake' in path_str or 'spoof' in path_str:
                label = 1

            if label is not None:
                data.append({'path': str(f), 'label': label})

    # Shuffle
    random.seed(CONFIG['SEED'])
    random.shuffle(data)

    df_temp = pd.DataFrame(data)
    if df_temp.empty: return df_temp

    # Balance and Limit
    real_df = df_temp[df_temp['label'] == 0]
    fake_df = df_temp[df_temp['label'] == 1]

    print(f"   > Found: Real={len(real_df)}, Fake={len(fake_df)}")

    min_len = min(len(real_df), len(fake_df))
    if min_len == 0: return df_temp

    limit_per_class = min(min_len, CONFIG['MAX_SAMPLES'] // 2)

    df_balanced = pd.concat([
        real_df.sample(limit_per_class),
        fake_df.sample(limit_per_class)
    ])
    return df_balanced.sample(frac=1).reset_index(drop=True)

df = scan_dataset(RAW_DIR)
print(f"📊 Final Dataset Size: {len(df)}")
print(df['label'].value_counts())

if len(df) < 100:
    raise RuntimeError("❌ Dataset download failed or is empty.")

# 4. PREPROCESSING
def compute_mels(row):
    path, label = row['path'], row['label']
    fname = Path(path).stem + "_" + str(random.randint(0,9999))
    save_path = MEL_DIR / f"{fname}.npy"

    try:
        y, sr = librosa.load(path, sr=CONFIG['SR'], duration=CONFIG['DURATION'])
        target_len = int(CONFIG['SR'] * CONFIG['DURATION'])
        if len(y) < target_len:
            y = np.pad(y, (0, target_len - len(y)))
        else:
            y = y[:target_len]

        mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=CONFIG['N_MELS'], hop_length=CONFIG['HOP_LEN'])
        mel_db = librosa.power_to_db(mel, ref=np.max)
        np.save(save_path, mel_db.astype(np.float16))
        return str(save_path), label
    except:
        return None, None

print("⚙️ Processing Audio (Wait 2-3 mins)...")
processed_data = [compute_mels(row) for _, row in tqdm(df.iterrows(), total=len(df))]
df_processed = pd.DataFrame([x for x in processed_data if x[0]], columns=['mel_path', 'label'])

# 5. MODEL
class VoiceDataset(Dataset):
    def __init__(self, df): self.df = df
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        mel = np.load(row['mel_path']).astype(np.float32)
        norm_mel = (mel - mel.mean()) / (mel.std() + 1e-6)
        return torch.tensor(norm_mel).unsqueeze(0), torch.tensor(row['label'])

train_size = int(0.8 * len(df_processed))
train_ds, val_ds = random_split(VoiceDataset(df_processed), [train_size, len(df_processed)-train_size])
train_loader = DataLoader(train_ds, batch_size=CONFIG['BATCH_SIZE'], shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=CONFIG['BATCH_SIZE'], shuffle=False)

class AudioDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1)),

            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        return self.net(x)

model = AudioDetector().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG['LR'])
criterion = nn.CrossEntropyLoss()
scaler = GradScaler('cuda')

# 6. TRAINING
print("🚀 Training Model...")
history = {'loss': [], 'val_acc': []}

for epoch in range(CONFIG['EPOCHS']):
    model.train()
    total_loss = 0
    for X, y in train_loader:
        X, y = X.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        with autocast('cuda'):
            loss = criterion(model(X), y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()

    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            all_preds.extend(model(X).argmax(1).cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    history['loss'].append(total_loss/len(train_loader))
    history['val_acc'].append(acc)
    print(f"Epoch {epoch+1} | Loss: {total_loss/len(train_loader):.4f} | Val Acc: {acc:.4f}")

# 7. RESULTS
plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)
plt.plot(history['loss'], label='Loss')
plt.plot(history['val_acc'], label='Accuracy')
plt.legend()
plt.title("Training Progress")

plt.subplot(1,2,2)
cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.title("Confusion Matrix")
plt.show()

: 

In [ ]:
# ==============================================================================
# 🚀 TEST YOUR TRAINED MODEL
# ==============================================================================
from google.colab import files
import librosa
import torch
import numpy as np

# 1. Upload
print("Upload an audio file (.wav or .mp3):")
uploaded = files.upload()
fname = list(uploaded.keys())[0]

# 2. Preprocess
def prepare_audio(path):
    y, sr = librosa.load(path, sr=16000, duration=4.0)
    target_len = 16000 * 4
    if len(y) < target_len: y = np.pad(y, (0, target_len - len(y)))
    else: y = y[:target_len]
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=80, hop_length=512)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    norm_mel = (mel_db - mel_db.mean()) / (mel_db.std() + 1e-6)
    return torch.tensor(norm_mel).unsqueeze(0).unsqueeze(0)

# 3. Predict
model.eval()
with torch.no_grad():
    tens = prepare_audio(fname).to(DEVICE)
    logits = model(tens)
    probs = torch.softmax(logits, dim=1)
    pred = torch.argmax(probs).item()

print("\n" + "="*30)
print(f"📁 File: {fname}")
print(f"🤖 Prediction: {'FAKE (AI)' if pred == 1 else 'REAL (Human)'}")
print(f"📊 Confidence: {probs[0][pred].item()*100:.2f}%")
print("="*30)

In [ ]:
# ==============================================================================
# 🚀 TEST YOUR TRAINED MODEL
# ==============================================================================
from google.colab import files
import librosa
import torch
import numpy as np

# 1. Upload
print("Upload an audio file (.wav or .mp3):")
uploaded = files.upload()
fname = list(uploaded.keys())[0]

# 2. Preprocess
def prepare_audio(path):
    y, sr = librosa.load(path, sr=16000, duration=4.0)
    target_len = 16000 * 4
    if len(y) < target_len: y = np.pad(y, (0, target_len - len(y)))
    else: y = y[:target_len]
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=80, hop_length=512)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    norm_mel = (mel_db - mel_db.mean()) / (mel_db.std() + 1e-6)
    return torch.tensor(norm_mel).unsqueeze(0).unsqueeze(0)

# 3. Predict
model.eval()
with torch.no_grad():
    tens = prepare_audio(fname).to(DEVICE)
    logits = model(tens)
    probs = torch.softmax(logits, dim=1)
    pred = torch.argmax(probs).item()

print("\n" + "="*30)
print(f"📁 File: {fname}")
print(f"🤖 Prediction: {'FAKE (AI)' if pred == 1 else 'REAL (Human)'}")
print(f"📊 Confidence: {probs[0][pred].item()*100:.2f}%")
print("="*30)

In [ ]:
# ==============================================================================
# 💾 EXPORT AS PYTORCH (.pth)
# ==============================================================================
from google.colab import files

# 1. Save the model weights
save_path = "audio_deepfake_detector.pth"
torch.save(model.state_dict(), save_path)
print(f"✅ Model saved to {save_path}")

# 2. Download to your local computer
files.download(save_path)